In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepc as qem

In [2]:
# we introduced noise into the network and trained it without any error mitigation techniques
# to observe the behavior of the Loss function degradation during training.

n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)

nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
pauli_set_of_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_of_n_qubits)

nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.9)
nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)

@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_without_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

def cost(x):
    return ((train_cir_without_qem(x))-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
steps = 100
params = weights

for i in range(steps):
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9594914
Cost after step    10:  0.2460653
Cost after step    15:  0.1239201
Cost after step    20:  0.0847992
Cost after step    25:  0.0668109
Cost after step    30:  0.0567241
Cost after step    35:  0.0503400
Cost after step    40:  0.0459657
Cost after step    45:  0.0428002
Cost after step    50:  0.0404181
Cost after step    55:  0.0385734
Cost after step    60:  0.0371133
Cost after step    65:  0.0359379
Cost after step    70:  0.0349792
Cost after step    75:  0.0341886
Cost after step    80:  0.0335310
Cost after step    85:  0.0329802
Cost after step    90:  0.0325158
Cost after step    95:  0.0321225
Cost after step   100:  0.0317879
Optimized rotation angles: [[[ 0.98527287  1.40624423  0.05604274]
  [ 0.24127555  0.76391887  0.1315054 ]]

 [[-0.11818263  2.3767891   0.52192344]
  [ 0.42481946  0.62236572  0.66514543]]]


In [3]:
#  It can be observed that after a certain number of iterations,
# the Loss function nearly stops decreasing,
# which is due to the limitation of the expressive capacity of the quantum network caused by noise.

In [4]:
# We estimate the total noise 'p_t' using the QEMZMSEPC method,
# allowing us to incorporate noise 'p_t' into the definition of the Loss function,
# thereby mitigating the impact of noise on quantum machine learning tasks.

operations = ['RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT',
              'RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT']

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

w = weights.copy().reshape(-1)

paras = [[0, w[0]], [0, w[1]], [0, w[2]], [0, w[3]], [0, w[4]], [0, w[5]], [0, 1], [1, 0],
         [0, w[6]], [0, w[7]], [0, w[8]], [0, w[9]], [0, w[10]], [0, w[11]], [0, 1], [1, 0]]
# Due to our assumption that the noise channel is time-invariant,
# we randomly initialize a quantum circuit to estimate,
# with the estimation of  considered to be stable and unchanged in the short term.

qemzmsepc = qem.QEMZMSEPC(n_qubits)
_, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                             kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
print(p_t)

0.8277125429510501


In [5]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_with_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

# We modify the definition of the Loss function
# by dividing the output of the quantum circuit by the noise parameter p_t
# to obtain the expected output of the quantum circuit after error mitigation,
# and then proceed with the Loss calculation.


def cost(x):
    return ((train_cir_with_qem(x)/p_t)-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
steps = 100
params = weights

for i in range(steps):
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.2788743
Cost after step    10:  0.0768806
Cost after step    15:  0.0333721
Cost after step    20:  0.0183003
Cost after step    25:  0.0114832
Cost after step    30:  0.0078555
Cost after step    35:  0.0057046
Cost after step    40:  0.0043276
Cost after step    45:  0.0033939
Cost after step    50:  0.0027323
Cost after step    55:  0.0022465
Cost after step    60:  0.0018795
Cost after step    65:  0.0015954
Cost after step    70:  0.0013712
Cost after step    75:  0.0011911
Cost after step    80:  0.0010442
Cost after step    85:  0.0009229
Cost after step    90:  0.0008215
Cost after step    95:  0.0007360
Cost after step   100:  0.0006632
Optimized rotation angles: [[[ 0.13852519  1.38179338  0.12677518]
  [ 0.39520627  1.44761074  0.23564253]]

 [[-0.21772258  1.67238086  0.31310511]
  [ 0.31703774  0.07002425  0.31852312]]]


In [6]:
# It can be seen that the Loss function after error mitigation performs better than without error mitigation.